# Dynamic Pricing for Urban Parking
**Model 2: Demand-Based Pricing Simulation using Pathway**

This notebook reads static parking lot data, applies dynamic pricing using a demand model, and writes the result to a downloadable CSV file.

In [ ]:
# Install Pathway if not already installed
!pip install -q pathway

In [ ]:
import pathway as pw

In [ ]:
# Define schema for the input CSV
class ParkingStreamSchema(pw.Schema):
    id: str
    systemcodenumber: str
    capacity: float
    latitude: float
    longitude: float
    occupancy: float
    vehicletype: str
    traffic: str
    queuelength: float
    isspecialday: float
    lastupdateddate: str
    lastupdatedtime: str

In [ ]:
# Read dataset (without header)
input_table = pw.io.csv.read(
    "dataset.csv",
    schema=ParkingStreamSchema,
    mode="static",
    has_header=False
)

In [ ]:
# Define Model 2 Pricing Logic
@pw.udf
def pricing_logic(occupancy, capacity, queuelength, traffic, isspecialday, vehicletype):
    vehicle_weights = {'car': 1.0, 'bike': 0.7, 'truck': 1.5, 'cycle': 0.6}
    traffic_weights = {'low': 0.5, 'average': 1.0, 'high': 1.5}
    try:
        demand = (
            1.5 * (occupancy / capacity) +
            0.5 * queuelength -
            traffic_weights.get(traffic.lower(), 1.0) +
            2.0 * isspecialday +
            vehicle_weights.get(vehicletype.lower(), 1.0)
        )
        norm_demand = min(1, max(0, demand / 10))
        price = 10 * (1 + 0.5 * norm_demand)
        return round(max(5, min(20, price)), 2)
    except:
        return 10.0

In [ ]:
# Apply pricing logic
result_table = input_table.select(
    lot_id=input_table.id,
    time=input_table.lastupdatedtime,
    model2_price=pricing_logic(
        input_table.occupancy,
        input_table.capacity,
        input_table.queuelength,
        input_table.traffic,
        input_table.isspecialday,
        input_table.vehicletype,
    )
)

In [ ]:
# Write result to CSV
pw.io.csv.write("final_output.csv", result_table)
pw.run()

In [ ]:
# Download the output CSV in Colab
from google.colab import files
files.download("final_output.csv")